In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
import numpy

### Analytical solution

$$y' = -y - x $$
Solve the complementary equation:
$$y' + y = 0$$
$$\frac{\mathrm{d}y}{\mathrm{d}x} = -y$$
$$\int \frac{\mathrm{d}y}{y} = -\int \mathrm{d}x$$
$$\ln y = -x + C_1$$
$$y = e^{-x+C_1}$$
$$y = C_2e^{-x}$$
Make a substitution: 
$$C_2 = u$$
Then:
$$y = ue^{-x}$$
$$y' = u'e^{-x} - ue^{-x}$$
Substitute:
$$u'e^{-x} = -x$$
$$\frac{\mathrm{d}u}{\mathrm{d}x} = -\frac{x}{e^{-x}}$$
$$\int \mathrm{d}u = \int -\frac{x{\mathrm{d}x}}{e^{-x}}$$
$$u = e^x(-x + 1) + C$$
$$y = Ce^{-x} - x + 1$$
Solve initial value problem for:
$$x_0 = 0,  y_0 = 1$$
$$1 = Ce^0 - 0 + 1$$
$$C = -1$$
The solution for IVP is:
$$y = -e^{-x} - x + 1$$

### Function definition

Let us define our function, its exact solution and method that will calculate exact solution for every given point.

In [2]:
def my_func(x, y):
    return -y - x

def exact_func(x, c):
    return c * numpy.exp(-x) - x + 1

def exact_on_space(x0, y0, X, number_of_steps):
    x = x_space(x0, X, number_of_steps)
    y = numpy.array([y0])
    c = (y0 + x0 - 1) / numpy.exp(-x0)
    
    for i in range(1, number_of_steps):
        y = numpy.append(y, exact_func(x[i], c))
        
    return y

### Helpers

Here we see some useful functions, which we will use pretty often.

In [3]:
# return array of given length filled with X values
def x_space(x0, X, number_of_steps):
    return numpy.linspace(x0, X, number_of_steps)

# init function for numberical methods
# return array of X, initial array of Y, const h
def helper(x0, y0, X, number_of_steps):
    return x_space(x0, X, number_of_steps), numpy.array([y0]), (X - x0) / float(number_of_steps)

### Euler method implementation

In [4]:
def euler(func, x0, y0, X, number_of_steps):
    x, y, h = helper(x0, y0, X, number_of_steps)
    
    for i in range(number_of_steps - 1): # -1 for reccurence
        y = numpy.append(y, y[i] + h * func(x[i], y[i]))
    
    return y

### Improved Euler method implementation

In [5]:
def euler_improved(func, x0, y0, X, number_of_steps):
    x, y, h = helper(x0, y0, X, number_of_steps)
    
    for i in range(number_of_steps - 1): # -1 for reccurence
        first = func(x[i], y[i])
        second = func(x[i + 1], y[i] + h * first)
        y = numpy.append(y, y[i] + (h * (first + second)) / 2)
        
    return y

### Runge-Kutta method implementation

In [6]:
def runge_kutta(func, x0, y0, X, number_of_steps):
    x, y, h = helper(x0, y0, X, number_of_steps)
    
    for i in range(number_of_steps - 1): # -1 for reccurence
        k1 = h * func(x[i], y[i])
        k2 = h * func(x[i] + h / 2, y[i] + k1 / 2)
        k3 = h * func(x[i] + h / 2, y[i] + k2 / 2)
        k4 = h * func(x[i] + h, y[i] + k3)
        y = numpy.append(y, y[i] + (k1 + 2 * k2 + 2 * k3 + k4) / 6)
    return y


### Global error calculation

Here we calculate global errors for 3 methods taking maximum from all.

In [7]:
def global_error(x0, y0, X, number_of_steps):
    y_euler_err = numpy.array([])
    y_euler_imp_err = numpy.array([])
    y_runge_err = numpy.array([])
    x_arr = list(range(number_of_steps))
    
    for x in x_arr:
        y_exact = exact_on_space(x0, y0, X, number_of_steps)
        y_euler = euler(my_func, x0, y0, X, number_of_steps)
        y_euler_imp = euler_improved(my_func, x0, y0, X, number_of_steps)
        y_runge = runge_kutta(my_func, x0, y0, X, number_of_steps)
        
        y_euler_err = numpy.append(y_euler_err, max(numpy.abs(y_exact - y_euler)))
        y_euler_imp_err = numpy.append(y_euler_imp_err, max(numpy.abs(y_exact - y_euler_imp)))
        y_runge_err = numpy.append(y_runge_err, max(numpy.abs(y_exact - y_runge)))
        
    return y_euler_err, y_euler_imp_err, y_runge_err

### Calculation block

Here we are doing main calculations to get desired values. Also here we get local errors.

In [8]:
x0 = 0
y0 = 1
X = 10
number_of_steps = 1000

x = x_space(x0, X, number_of_steps)

y_exact = exact_on_space(x0, y0, X, number_of_steps)
y_euler = euler(my_func, x0, y0, X, number_of_steps)
y_euler_imp = euler_improved(my_func, x0, y0, X, number_of_steps)
y_runge = runge_kutta(my_func, x0, y0, X, number_of_steps)

# local errors
euler_local_err = numpy.abs(y_exact - y_euler)
euler_imp_local_err = numpy.abs(y_exact - y_euler)
runge_local_err = numpy.abs(y_exact - y_runge)

global_euler_err, global_euler_imp_err, global_runge_err = global_error(x0, y0, X, number_of_steps)

In [9]:
output_notebook()

Loading BokehJS ...

### Plotting

In [10]:
p1 = figure(title="Numerical methods", x_axis_label='x', y_axis_label='y')
a = p1.line(x, y_exact, legend="Exact solution", line_width=2, line_color="#FF0000")
b = p1.line(x, y_euler, legend="Euler method", line_width=2, line_color="#00FF00")
c = p1.line(x, y_euler_imp, legend="Improved Euler method", line_width=2, line_color="#0000FF")
d = p1.line(x, y_runge, legend="Runge-Kutta method", line_width=2, line_color="#FF00FF")

p2 = figure(title="Global errors of methods", x_axis_label='step', y_axis_label='Global error')
e = p2.line(x, global_euler_err, legend="Error of Euler", line_width=2, line_color="#FF0000")
f = p2.line(x, global_euler_imp_err, legend="Error of Improved Euler", line_width=2, line_color="#00FF00")
g = p2.line(x, global_runge_err, legend="Error of Runge-Kutta", line_width=2, line_color="#0000FF")

p3 = figure(title="Local errors of methods", x_axis_label='x', y_axis_label='Local error')
h = p3.line(x, euler_local_err, legend="Error of Euler", line_width=2, line_color="#FF0000")
i = p3.line(x, euler_imp_local_err, legend="Error of Improved Euler", line_width=2, line_color="#00FF00")
j = p3.line(x, runge_local_err, legend="Error of Runge-Kutta", line_width=2, line_color="#0000FF")

p1.legend.location = "bottom_left"
p2.legend.location = "top_right"
p1.legend.click_policy = "hide"
p2.legend.click_policy = "hide"
p3.legend.click_policy = "hide"


### Finally, graphs.

There are few things to be mentioned.
1. With first look on graph we think that we made a mistake. But the graph is correct, it's just plots which are the same.
2. We can see that both Euler methods are overcoming Runge-Kutta.
3. Local error becomes bigger as X - $x_0$ becomes bigger.

In [11]:
show(p1, notebook_handle=True)
show(p2, notebook_handle=True)
show(p3, notebook_handle=True)